In [24]:
!pip install torch lightning numpy kaggle wandb torch-geometric torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 2.3 MB/s eta 0:00:00


In [39]:
!pip install kaggle -U
!pip install  lightning -U
!pip install torch  -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 944.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.1-py3-none-any.whl size=111904 sha256=aeb489c34723fcdb250f5a3a80ab373c2c30267ee15bbb79feb500594b50cdb5
  Stored in directory: /root/.cache/pip/wheels/a0/83/bd/ef72ad59f675b6bc41e6708814eae8e4b9ae28a5401439fe4b
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 725.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexxxyy47","key":"29be390563ab164c9d7671a10a031097"}'}

In [4]:
#!rm -Rf ~/.kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip
!rm paysim1.zip

  0% 0.00/178M [00:00<?, ?B/s]
  0% 0.00/178M [00:00<?, ?B/s]
Archive:  paysim1.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of paysim1.zip or
        paysim1.zip.zip, and cannot find paysim1.zip.ZIP, period.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/uni/paysim1.zip .
!unzip paysim1.zip
!rm paysim1.zip

Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [50]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as L, wandb as wndb, pdb
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.utils import shuffle
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from matplotlib import pyplot as plt
import networkx as nx
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch import optim
from torchmetrics.classification import BinaryF1Score
from torch_geometric.nn import MessagePassing
from typing import Optional
from torch.nn import Parameter
from torch_geometric.typing import (
    Adj,
    OptPairTensor,
    OptTensor,
    SparseTensor,
    torch_sparse,
)
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.utils import add_self_loops as add_self_loops_fn
from torch_geometric.utils import (
    is_torch_sparse_tensor,
    scatter,
    spmm,
    to_edge_index,
)
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils.sparse import set_sparse_value
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros
from torch.nn.functional import binary_cross_entropy_with_logits
from sklearn.preprocessing import StandardScaler
from enum import Enum
from torcheval.metrics.functional import binary_f1_score
from torchmetrics.functional.classification import binary_accuracy


In [40]:
# PARAMETERS

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ACCELERATOR =  "gpu" if torch.cuda.is_available() else "cpu"
SEED = 42
PRECISION = torch.float

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEPTH = 3
LR = 1e-3
WD = 1e-5
H_SIZE = 32
BATCH_SIZE = 2048

"aaf831dabc88d936d4e6b439b798bb4cb42814ea"

#wndb.login()



'aaf831dabc88d936d4e6b439b798bb4cb42814ea'

In [41]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pd.read_csv(dataset_file)


def find_null_or_empty_records( dataframe: pd.DataFrame):
    n = len(dataframe)
    for index, row in dataframe.iterrows():
        print_progress_bar(index/n)
        # Controlla se ci sono valori nulli o vuoti nel record
        if row.isnull().any() or any(map(lambda x: x == '', row)):
            # Stampa il record
            print(f"Record con valori nulli o vuoti:\n{row}\n")

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.query('abs(oldbalanceOrg - newbalanceOrig) != amount'))/len(dataframe),
            "inconsistent dest balance": len(dataframe.query('abs(oldbalanceDest - newbalanceDest) != amount'))/len(dataframe),
            "zero cash transaction": len(dataframe.query('amount == 0 '))/len(dataframe),
            "self-transaction": len(dataframe.query('nameOrig == nameDest'))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe['type'].value_counts().reset_index()

    # Rinomina le colonne
    counts.columns = ['type', 'count']

    counts['count'] = counts['count'] / counts['count'].sum()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()


def build_names_conversion_index(df):
  all_names = pd.concat([df['nameOrig'], df['nameDest']])


  # Applica factorize per ottenere identificatori univoci
  unique_names, name_indices = pd.factorize(all_names)


  # Costruisci il dizionario di mapping
  name_to_id = dict(zip(name_indices, unique_names))
  #breakpoint()
  return name_to_id


class DatasetType(Enum):
  TRAIN = 0,
  VALIDATION = 1,
  TEST = 2



def create_weights_tensor(original_tensor, n, m):
    # Crea un nuovo tensore con la stessa forma del tensore

    new_tensor = torch.full_like(original_tensor, m)

    # Imposta i valori a n dove il tensore originale ha valore 1
    new_tensor[original_tensor == 1] = n

    return new_tensor



In [42]:
class FraudDetectionDataset(Dataset):

    def __init__(self,dataset_file : str, kind):
        dataframe = load_dataframe(dataset_file)
        dataframe["isFraud"] = dataframe["isFraud"].astype(int)
        dataframe["isFlaggedFraud"] = dataframe["isFlaggedFraud"].astype(int)
        train_size = int(0.7 * len(dataframe))
        val_size = int(0.1 * len(dataframe))
        test_size = len(dataframe) - train_size - val_size

        indices = list(range(len(dataframe)))
        train_indices, val_indices, test_indices = random_split(indices, [train_size, val_size, test_size])

        # Crea i DataFrame per i set di addestramento, convalida e test
        train_set = dataframe.iloc[list(train_indices)]
        val_set = dataframe.iloc[list(val_indices)]
        test_set = dataframe.iloc[list(test_indices)]
        if kind == DatasetType.TRAIN:
          self.raw_data = train_set
        elif kind == DatasetType.VALIDATION:
          self.raw_data = val_set
        else:
          self.raw_data = test_set
        self.raw_data = pd.DataFrame(self.raw_data)

        self.transaction_types = {
            "CASH_IN": 0,
            "CASH_OUT": 1,
            "DEBIT": 2,
            "PAYMENT": 3,
            "TRANSFER": 4
        }

    def analize_data(self,find_empty_records = True):
        print("----HEAD----")
        print(self.raw_data.head())
        print("----INFO----")
        print(self.raw_data.info())
        print("----DESCRIBE----")
        print(self.raw_data.describe())
        if find_empty_records:
          find_null_or_empty_records(self.raw_data)

    def extract_inconsistent_transactions(self):
        condiction = "abs(oldbalanceOrg - newbalanceOrig) != amount | abs(oldbalanceDest - newbalanceDest) != amount | amount == 0 | nameOrig == nameDest"

        return self.raw_data.query(condiction)


    def remove_inconsistent(self):
        self.raw_data = self.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) == abs(oldbalanceDest - newbalanceDest) & amount != 0")


    def convert_type(self):
      scaler = StandardScaler()
      self.raw_data['type'] = self.raw_data['type'].apply(lambda x: self.transaction_types[x])
      self.raw_data['step'] = self.raw_data['step'].apply(lambda x: x%24)
      #breakpoint()
      mean = self.raw_data['oldbalanceOrg'].mean()
      self.raw_data['oldbalanceOrg'] = self.raw_data['oldbalanceOrg'].apply(lambda x: x/mean)
      mean = self.raw_data['oldbalanceDest'].mean()
      self.raw_data['oldbalanceDest'] = self.raw_data['oldbalanceDest'].apply(lambda x: x/mean)
      # self.raw_data['oldbalanceOrg'] = scaler.fit_transform(self.raw_data['oldbalanceOrg'])
      # self.raw_data['oldbalanceDest'] = scaler.fit_transform(self.raw_data['oldbalanceDest'].values.reshape(-1, 1))




    def build_graph(self, data):



      edge_index = []
      #breakpoint()
      node_names = build_names_conversion_index(data)
      id = 0
      x = []
      data['nameOrig'] = data['nameOrig'].apply(lambda x: node_names[x])
      data['nameDest'] = data['nameDest'].apply(lambda x: node_names[x])
      ordered_items = sorted(list(node_names.items()), key=lambda x: x[1])
      for node in ordered_items:
        x.append([1 if node[0].startswith('M') else 0])

      edge_attr = torch.tensor(data[['step', 'type', 'amount', 'oldbalanceOrg', 'oldbalanceDest']].values, dtype=PRECISION)
      y = torch.tensor(data['isFraud'].to_numpy(), dtype=PRECISION)
      edge_index = torch.tensor(data[['nameOrig', 'nameDest']].values, dtype=torch.int).t()
      x = torch.tensor(x, dtype=PRECISION)

      #breakpoint()
      data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
      data.y = y
      return data


    def __getitem__(self, index):
      return self.raw_data.iloc[index]

    def __len__(self):
      return len(self.raw_data)


    def collate_train(self, data):

      #breakpoint()
      return self.build_graph(pd.DataFrame(data))

    def collate_val(self, data):

      #breakpoint()
      return data[0], self.val_ids



    def get_dataloader(self,batch_size, validation=False):
      return DataLoader(self, batch_size=batch_size, shuffle=False, collate_fn = self.collate_train)


    def to_device(self):
      #breakpoint()
      self.data[0].x.to(DEVICE)
      self.data[0].y.to(DEVICE)
      self.data[0].edge_attr.to(DEVICE)
      self.data[0].edge_index.to(DEVICE)
      self.data[0].train_mask.to(DEVICE)
      self.data[0].val_mask.to(DEVICE)
      self.data[0].test_mask.to(DEVICE)









In [8]:
dataset = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv")
#inconsistent_data = dataset.extract_inconsistent_transactions()

In [ ]:
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)
names_distinct = set(dataset.raw_data["nameOrig"]).union(set(dataset.raw_data["nameDest"]))

In [ ]:
print(dataset.raw_data.columns)
dataset.convert_type()
dataset.remove_inconsistent()
dataset.build_graph(0.1,0.2)




Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


In [ ]:
def convert_to_networkx(graph):
	g = to_networkx(graph, node_attrs=['x'])

	return g

def plot_graph(g):
	plt.figure(figsize=(9,7))
	nx.draw_spring(g, node_size=30, arrows=False)
	plt.show()



g = convert_to_networkx(dataset.data[0])
plot_graph(g)
del g


In [55]:
class FraudDetectionModule(L.LightningModule):

  def __init__(self,nodes_size,edges_size, hidden_layer_size_gnn,hidden_layer_size_edges , depth_gnn, depth_edges, lr=1e-3, wd=5e-4):
    super().__init__()
    self.layers = []

    self.layers.append(GCNConv(nodes_size, hidden_layer_size_gnn).to(DEVICE))
    for _ in range(depth_gnn - 2):
      self.layers.append(GCNConv(hidden_layer_size_gnn, hidden_layer_size_gnn).to(DEVICE))


    if hidden_layer_size_edges < 2:
      self.edges_nn = nn.Linear(edges_size,1)
    else:
      edges_layers = [nn.Linear(edges_size,hidden_layer_size_edges),nn.ELU()]
      for _ in range(depth_edges-2):
        edges_layers.append(nn.Linear(hidden_layer_size_edges, hidden_layer_size_edges))
        edges_layers.append(nn.ELU())
      edges_layers.append(nn.Linear(hidden_layer_size_edges,1))
      self.edges_nn = nn.Sequential(*edges_layers)


    self.classifier = nn.Linear(2*hidden_layer_size_gnn+5, 1)
    self.relu = nn.ELU()
    self.sigmoid = nn.Sigmoid()
    self.lr = lr
    self.wd = wd
    self.training_step_outputs = []
    self.validation_step_outputs = []
    self.validation_f1 = []
    self.f1 = BinaryF1Score()
    self.automatic_optimization = False



    self.loss = nn.BCEWithLogitsLoss()


  def forward(self,data):
    #breakpoint()
    edge_index = data.edge_index
    edge_attr = data.edge_attr# nn.functional.normalize(data[0].edge_attr) #data.edge_attr
    x = data.x



    if torch.isinf(edge_index).any().item():
      print("edge_index_inf")
      exit(1)
    if (torch.isnan(edge_index ).any().item()):
      print("edge_index")
      exit(1)
    if torch.isinf(edge_attr).any().item():
      print("edge_attr_inf 1")
      exit(1)
    if (torch.isnan(edge_attr ).any().item()):
      print("edge_attr 1")
      exit(1)
    if torch.isinf(x).any().item():
      print("x_inf 1")
      exit(1)
    if (torch.isnan(x ).any().item()):
      print("x 1")
      exit(1)

    #breakpoint()
    #print(edge_attr)
    #print(self.edges_nn[0].weight)

    #edge_attr = self.edges_nn(edge_attr)
    if torch.isinf(edge_attr).any().item():
      print("edge_attr_inf 2")
      exit(1)
    if (torch.isnan(edge_attr ).any().item()):

      print("edge_attr 2")
      print(edge_attr)
      exit(1)

    for cnn in self.layers:
      x = self.relu(cnn(x, edge_index))
    # if (torch.isnan(x ).any().item()):
    #   print("x 2")
    #   exit(1)



    #add batch  with edge nodes + features of edges
    from_nodes = torch.nan_to_num(x.squeeze()[edge_index[0,:].squeeze()])
    dest_nodes = torch.nan_to_num(x.squeeze()[edge_index[1,:].squeeze()])

    to_classify = torch.cat((from_nodes,dest_nodes,edge_attr),dim=1)
    if torch.isinf(to_classify).any().item():
      print("to_classify_inf 2")
      exit(1)
    if (torch.isnan(to_classify ).any().item()):
      print("to_classify")
      exit(1)
    del from_nodes
    del dest_nodes
    return self.sigmoid(self.classifier(to_classify).squeeze())



  def training_step(self, batch, batch_idx):
    #breakpoint()
    opt = self.optimizers()
    opt.zero_grad()
    #print("training")

    z = self.forward(batch)

    #breakpoint()
    w = create_weights_tensor(batch.y,1,0.7)
    # z[z >= 0.5] = 1
    # z[z < 0.5] = 0
    # loss = torch.nn.functional.binary_cross_entropy(z,batch.y)
    loss = binary_cross_entropy_with_logits(z,batch.y,weight=w)
    #loss.backward(retain_graph=True)

    #print("Gradiente del primo parametro:", self.edges_nn[0].weight.grad)
    #print("train_loss = ",loss)
    # Logging to TensorBoard (if installed) by default
    self.manual_backward(loss)
    #print("Gradiente del primo parametro:", self.edges_nn[0].weight.grad)
    self.training_step_outputs.append(loss)
    self.log("train_loss", loss, prog_bar=True)
    self.clip_gradients(opt, gradient_clip_val=0.5, gradient_clip_algorithm="norm")
    opt.step()


  def on_train_epoch_end(self):
        all_preds = torch.stack(self.training_step_outputs)
        # do something with all preds
        print("avg train loss = ", all_preds.mean())
        self.training_step_outputs.clear()


  def validation_step(self, batch, batch_idx):
    #breakpoint()

    #print("validation")

    z = self.forward(batch)
    #print(list(z))
    #print(z.shape)
    breakpoint()
    val_loss = self.loss(z,batch.y)
    self.validation_step_outputs.append(val_loss)
    #print("val_loss = ", float(val_loss))
    f1 = binary_f1_score(z, batch.y) #self.f1(z,batch.y)
    acc = binary_accuracy(z, batch.y)
    print("acc = ",acc)
    self.validation_f1.append(f1)
    #print("f1-score = ", float(f1))
    #wndb.log({"val_loss": val_loss,"f1-score":f1})
    self.log_dict({"val_loss": val_loss,"f1-score":f1}, prog_bar=True)

  def on_validation_epoch_end(self):
        all_preds = torch.stack(self.validation_step_outputs)
        f1s = torch.stack(self.validation_f1)

        # do something with all preds
        print("avg val loss = ", all_preds.mean())
        print("avg val f1 = ", f1s.mean())
        self.validation_step_outputs.clear()
        self.validation_f1.clear()


  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)
    return optimizer




In [44]:
dataset_train = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv",DatasetType.TRAIN)
dataset_validation = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv",DatasetType.VALIDATION)
dataset_train.convert_type()
dataset_train.remove_inconsistent()
dataset_validation.convert_type()
dataset_validation.remove_inconsistent()
# wndb.init(
#     project="datamining-hw4",

print("train len ",len(dataset_train.raw_data))
print("val len ",len(dataset_validation.raw_data))

#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR ,
#     "weight decay": WD ,
#     "layers": DEPTH ,
#     "hidden layers size": H_SIZE
#     })train_size = int(0.8 * len(dataset))

print(DEVICE)
#dataset.to_device()
train_loader = dataset_train.get_dataloader(BATCH_SIZE)
val_loader = dataset_validation.get_dataloader(BATCH_SIZE)
#breakpoint()

#model.to(DEVICE)

trainer = L.Trainer(deterministic=True,max_epochs=30,accelerator=ACCELERATOR)



INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


train len  542151
val len  77525
cpu


In [56]:

model = FraudDetectionModule(1,5 ,H_SIZE,H_SIZE,DEPTH,DEPTH,LR,WD)

In [57]:
# for name, param in model.named_parameters():
    # print(f"Parameter {name} device:", param.device)



trainer.fit(model, train_loader, val_loader)
#wndb.finish() 1131626


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /content/lightning_logs/version_6/checkpoints exists and is not empty.
INFO: 
  | Name       | Type              | Params
-------------------------------------------------
0 | edges_nn   | Sequential        | 1.3 K 
1 | classifier | Linear            | 70    
2 | relu       | ELU               | 0     
3 | sigmoid    | Sigmoid           | 0     
4 | f1         | BinaryF1Score     | 0     
5 | loss       | BCEWithLogitsLoss | 0     
-------------------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.005     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | edges_nn   | Sequential        | 1.3 K 
1 | classifier | Linear            | 70    
2 | relu       | ELU               | 0  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.

PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



> <ipython-input-55-654a7b8568b7>(148)validation_step()
    146     #print(z.shape)
    147     breakpoint()
--> 148     val_loss = self.loss(z,batch.y)
    149     self.validation_step_outputs.append(val_loss)
    150     #print("val_loss = ", float(val_loss))

ipdb> c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 347, in set_continue
    sys.settrace(None)



...

Validation: |          | 0/? [00:00<?, ?it/s]

> <ipython-input-55-654a7b8568b7>(148)validation_step()
    146     #print(z.shape)
    147     breakpoint()
--> 148     val_loss = self.loss(z,batch.y)
    149     self.validation_step_outputs.append(val_loss)
    150     #print("val_loss = ", float(val_loss))

ipdb> n
> <ipython-input-55-654a7b8568b7>(149)validation_step()
    147     breakpoint()
    148     val_loss = self.loss(z,batch.y)
--> 149     self.validation_step_outputs.append(val_loss)
    150     #print("val_loss = ", float(val_loss))
    151     f1 = binary_f1_score(z, batch.y) #self.f1(z,batch.y)

ipdb> n
> <ipython-input-55-654a7b8568b7>(151)validation_step()
    149     self.validation_step_outputs.append(val_loss)
    150     #print("val_loss = ", float(val_loss))
--> 151     f1 = binary_f1_score(z, batch.y) #self.f1(z,batch.y)
    152     acc = binary_accuracy(z, batch.y)
    153     print("acc = ",acc)

ipdb> binary_f1_score(z, batch.y)


tensor(0.)
ipdb> batch.y
tensor([0., 0., 0.,  ..., 0., 0., 0.])
ipdb> batch.y[batch.y == 1]
tensor([])



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 361, in set_quit
    sys.settrace(None)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/debugger.py", line 1075, in cmdloop
    sys.settrace(None)



--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
acc =  tensor(1.)
> <ipython-input-55-654a7b8568b7>(148)validation_step()
    146     #print(z.shape)
    147     breakpoint()
--> 148     val_loss = self.loss(z,batch.y)
    149     self.validation_step_outputs.append(val_loss)
    150     #print("val_loss = ", float(val_loss))

ipdb> q
ipdb> q


In [ ]:
dataset.analize_data(find_empty_records=False)
print("inconsistent fraud")
print(len(inconsistent_data.query('isFraud == 1'))/len(inconsistent_data))
print("inconsistent not fraud")
print(len(inconsistent_data.query('isFraud == 0'))/len(inconsistent_data))
print("given > gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) > abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("given < gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) < abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("distinct origin name")
print(dataset.raw_data["nameOrig"].nunique())
print("distinct names")
print(len(names_distinct))
print("isFlaggedFraud and isFraud")
print(len(dataset.raw_data.query("isFlaggedFraud & isFraud"))/len(dataset.raw_data.query("isFraud")))
plot_categories(dataset.raw_data)
